In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [2]:
text = open("input.txt", 'r').read()
# print(text[:200])

In [10]:
vocab = sorted(list(set(''.join(text))))
# print(''.join(vocab))

stoi = {s:i for i,s in enumerate(vocab)}
itos = {i:s for i,s in enumerate(vocab)}

encode = lambda s: [stoi[ch] for ch in s]
# print(encode('hi there!'))

decode = lambda l: ''.join([itos[i] for i in l])
# print(decode(encode('hi there!')))

itos

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [7]:
data = torch.tensor(encode(text))
print(text[:100])
print(data[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8
train_data[:block_size+1]

# a sequence of block_size+1 tokens would become block_size number of examples

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
# visualizing data samples from a chunk of data

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} \t target is {target}')

when input is tensor([18]) 	 target is 47
when input is tensor([18, 47]) 	 target is 56
when input is tensor([18, 47, 56]) 	 target is 57
when input is tensor([18, 47, 56, 57]) 	 target is 58
when input is tensor([18, 47, 56, 57, 58]) 	 target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) 	 target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) 	 target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) 	 target is 58


In [21]:
# introducing batch dimension
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([train_data[i:i+block_size] for i in ix])
    y = torch.stack([train_data[i+1:i+1+block_size] for i in ix])

    return x,y

xb,yb = get_batch('train')
print("inputs: ")
print(xb)
print(xb.shape)
print("----------\n")
print("targets: ")
print(yb)
print(yb.shape)

inputs: 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
----------

targets: 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
torch.Size([4, 8])


In [ ]:
# visualize how the data samples would look like?

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context} \t target is {target}')

# Bigram LM

In [22]:
class BigramLM(nn.Module):
    pass